## Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Increase data frame column width:
pd.set_option('max_colwidth', 400)

import re

# for tokenization
from sklearn.feature_extraction.text import CountVectorizer
# natto-py
import natto
from natto import MeCab

# for sentiment analysis
# oseti
import oseti
# asari
from asari.api import Sonar

# deep-translator, interfacing with Google Translate
# for japanese-english translation
from deep_translator import GoogleTranslator

# VADER
# for english sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import os
# this explicitly tells python where to find the MeCab dictionary
# replace with the path to the libmecab.dll if you get an error running the MeCab() instantiation below
os.environ['MECAB_PATH']="C:/Program Files/MeCab/bin/libmecab.dll"
# we need to force MeCab to read Japanese text in UTF-8
os.environ['MECAB_CHARSET']='utf-8'

### Comparing Japanese Sentiments and Translated English Sentiments

In this section, we compare how the Japanese sentiment analyzers on Japanese reviews perform against the English sentiment analyzers on Japanese reviews translated to English.

For the purposes of transparency, the translator we shall use is Google Translate from the `deep-translator` library, available [here](https://pypi.org/project/deep-translator/).

First, we need to initialize our analyzers. For Japanese, we will be using `asari`, and for English, we will be using VADER's sentiment analyzer.

In [3]:
# japanese sentiment analyzer - asari
analyzer_ja = Sonar()

# english sentiment analyzer - VADER
analyzer_en = SentimentIntensityAnalyzer()

We also need to initialize our translator. This will be an API for Google Translate.

In [4]:
# Japanese source to English target
translator = GoogleTranslator(source='ja', target='en')

We also introduce our test subject here - a review in the Japanese language.

In [5]:
test_review_1_ja = '【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）。地元の方にも人気のある温泉のようで夕方は混み合う。そして顔見知りの方々の会話が長く続いている。マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）。また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない。改善されれば幸いです。'


This can be translated to English using Google Translate as well.

In [6]:
test_review_1_en = translator.translate(test_review_1_ja)
test_review_1_en

"[Good points] - The view from the room was wonderful and I was very satisfied. - Polite reception at the front desk. [Points that bothered me] - I didn't know that all rooms were non-smoking (although I didn't confirm this myself) There was no mention of non-smoking or smoking when I made the reservation, so I didn't worry about it, but there was no ashtray in the room, so I asked the front desk. When I asked over the phone, I was told that all rooms are non-smoking. I had to go outside the facility, so I couldn't really relax. - Hair in the room There are two long hairs on the white sofa. The sofa was white so it stood out.・Food quality and service The quality of the food was average and the quantity was just right, but I felt that the quality was not commensurate with the price of the room. At the meal venue, all staff members were in the kitchen, and there were many hours when no one was in the venue. I wanted to add another drink, but I had to wait about 5 minutes for the person i

As we can see, Google Translate does a pretty good job at translating the text. Let's see how much better VADER fares compared to `asari`.

In [7]:
# japanese sentiment
analyzer_ja.ping(test_review_1_ja)

{'text': '【良かった点】・部屋から見える景色が素晴らしく大満足。・フロントの対応が丁寧。【気になった点】・全室禁煙を知らなかった（当方の確認不足もあるが）予約時に禁煙や喫煙の記載がなかったので気にしてなかったが、部屋に灰皿がないのでフロントに電話で聞いたところ、全室禁煙とのこと。施設外まで出る必要があり、イマイチのんびり出来なかった。・部屋の髪の毛白いソファーに長い髪の毛が2本。ソファーが白なので目立ちました。・食事の質とサービス食事の質は普通であり、量もちょうどよかったが、質は宿泊料金に見合ってないように感じた。食事会場では係の方全員が厨房側に入り、会場に誰もいない時間が多々あった。飲み物を追加したいが、係の人が来るまで5分程度待った（大声で呼ぶことも考えたが、周囲のお客様に迷惑と思った）。朝食は席に着いても説明がなく戸惑った。・感染対策浴場と脱衣所でマスクなしでの会話が目立つ（地元の日帰り客のようです）。地元の方にも人気のある温泉のようで夕方は混み合う。そして顔見知りの方々の会話が長く続いている。マスク無しでの会話について、ホテル側から特にお願いはないようです（張り紙とか何もなかった）。また、夕食、朝食会場の席が部屋番号ごとに指定されていたが、使ってない席もあるのに分散させていない。改善されれば幸いです。',
 'top_class': 'negative',
 'classes': [{'class_name': 'negative', 'confidence': 0.5721551775932312},
  {'class_name': 'positive', 'confidence': 0.4278448224067688}]}

In [8]:
# english sentiment
analyzer_en.polarity_scores(test_review_1_en)

{'neg': 0.059, 'neu': 0.893, 'pos': 0.048, 'compound': -0.6075}

Following the process of assigning a sentiment score in EDA part 1, our Japanese analyzer would assign it a score of around -0.15, which is significantly less negative than the -0.6075 that VADER gives. 

Let's try it again with another review - this time looking at the review that was almost entirely on the room.

In [9]:
test_review_2_ja = 'お部屋おまかせで予約をしました。部屋のドアを開けると凄く広い部屋でした。洗面所の前に椅子が2つありました。ひとつは普通にビジホにある様な椅子で、もうひとつは介護が必要な人が使うような椅子。それをみて違和感をおぼえました。部屋の中やトイレ、お風呂をじっくりみてみるとバリアフリーの部屋でした。まるで介護施設か病院の個室にいるような感じでした。約10日間の旅行の内の1日だけのホテルでしたが、旅行の楽しい気分が台無しになりました。その椅子を見ると気分悪くなるので見えない入口に移動させました。お風呂も入る気になれなかった。年齢が70超えているのなら納得しますがそこまで老いていないのにこの部屋。めちゃくちゃ居心地悪かった。気分だだ下がり。後、ベットの上にナイトウエアバスタオル、スリッパ等置いてあるのも違和感です。バスタオルがベッドの上にあるとうっかりお風呂入ってしまうと濡れた状態でバスタオル取りに行く感じになります。前にも同じようなクチコミがありましたが全く改善されていません。後もう1つコーヒー100円納得いかない。ウエルカムドリンクでアルコールまで無料で提供するホテルもあるのに100円でコーヒー買うなんてほんと有り得ない。これも前のクチコミにありました。とにかく不愉快なホテルでした。とクチコミを投稿しても都合の悪いクチコミには返信もないのですね。見て見ぬふりですか'
test_review_2_en = translator.translate(test_review_2_ja)
test_review_2_en

"I made a reservation for the room. When I opened the door to the room, it was a very large room. There were two chairs in front of the washroom. One was a chair like the one you would find at a business hotel, and the other was a chair used by people who needed nursing care. I felt a sense of discomfort when I saw that. When I took a closer look at the room, toilet, and bath, I found that it was a barrier-free room. It felt like I was in a private room at a nursing home or hospital. I only stayed at the hotel for one day out of a 10-day trip, but it ruined the fun of the trip. I felt sick when I saw that chair, so I moved it to an entrance where I couldn't see it. I didn't even feel like taking a bath. It would make sense if she was over 70 years old, but she's not that old in this room. It was extremely uncomfortable. I feel depressed. Also, it felt strange to have nightwear, bath towels, slippers, etc. on the bed. If you accidentally take a bath and your bath towel is on the bed, yo

In [10]:
[sentence for sentence in test_review_2_en.split('. ')]

['I made a reservation for the room',
 'When I opened the door to the room, it was a very large room',
 'There were two chairs in front of the washroom',
 'One was a chair like the one you would find at a business hotel, and the other was a chair used by people who needed nursing care',
 'I felt a sense of discomfort when I saw that',
 'When I took a closer look at the room, toilet, and bath, I found that it was a barrier-free room',
 'It felt like I was in a private room at a nursing home or hospital',
 'I only stayed at the hotel for one day out of a 10-day trip, but it ruined the fun of the trip',
 "I felt sick when I saw that chair, so I moved it to an entrance where I couldn't see it",
 "I didn't even feel like taking a bath",
 "It would make sense if she was over 70 years old, but she's not that old in this room",
 'It was extremely uncomfortable',
 'I feel depressed',
 'Also, it felt strange to have nightwear, bath towels, slippers, etc',
 'on the bed',
 'If you accidentally tak

In [11]:
# japanese sentiment
analyzer_ja.ping(test_review_2_ja)

{'text': 'お部屋おまかせで予約をしました。部屋のドアを開けると凄く広い部屋でした。洗面所の前に椅子が2つありました。ひとつは普通にビジホにある様な椅子で、もうひとつは介護が必要な人が使うような椅子。それをみて違和感をおぼえました。部屋の中やトイレ、お風呂をじっくりみてみるとバリアフリーの部屋でした。まるで介護施設か病院の個室にいるような感じでした。約10日間の旅行の内の1日だけのホテルでしたが、旅行の楽しい気分が台無しになりました。その椅子を見ると気分悪くなるので見えない入口に移動させました。お風呂も入る気になれなかった。年齢が70超えているのなら納得しますがそこまで老いていないのにこの部屋。めちゃくちゃ居心地悪かった。気分だだ下がり。後、ベットの上にナイトウエアバスタオル、スリッパ等置いてあるのも違和感です。バスタオルがベッドの上にあるとうっかりお風呂入ってしまうと濡れた状態でバスタオル取りに行く感じになります。前にも同じようなクチコミがありましたが全く改善されていません。後もう1つコーヒー100円納得いかない。ウエルカムドリンクでアルコールまで無料で提供するホテルもあるのに100円でコーヒー買うなんてほんと有り得ない。これも前のクチコミにありました。とにかく不愉快なホテルでした。とクチコミを投稿しても都合の悪いクチコミには返信もないのですね。見て見ぬふりですか',
 'top_class': 'negative',
 'classes': [{'class_name': 'negative', 'confidence': 0.5619028210639954},
  {'class_name': 'positive', 'confidence': 0.4380972385406494}]}

In [12]:
# english sentiment
analyzer_en.polarity_scores(test_review_2_en)

{'neg': 0.14, 'neu': 0.797, 'pos': 0.063, 'compound': -0.9793}

This is surprisingly rather accurate to the actual sentiment of the sentence, though we would need to perform a more complete statistical analysis in order to confirm it.

In fact, let us populate seven more sentiment scores, this time by translating each sentence into English first, then running it through VADER.

In [13]:
# aspect categories
aspect_words = {
    'service': ['サービス', 'スタッフ', 'フロント', 'チェックイン', '丁寧', '親切', '接客', 'サーバ'],
    'location': ['立地', '駅', 'バス', '近く', '便利', '駐車', 'コンビニ', '場所'],
    'room': ['部屋', '広い', '宿泊', 'ベッド', '値段'], 
    'amenities': ['アメニティ', '無料'],
    'bathroom': ['風呂', '温泉', '浴場', '露天風呂', '清潔', '湯', 'トイレ'],
    'food': ['朝食', '食事', '料理', '夕食', 'バイキング', 'メニュー', 'ご飯', '酒', '飲']
}

In [14]:
def has_aspect(aspect: str, text: str):
    '''Returns a boolean describing whether a document contains an aspect word or not
    '''
    # this returns True if it exists in the list
    # otherwise returns False
    return max([(word in text) for word in aspect_words[aspect]])

In [15]:
# assigning aspects to each sentence in the review that has any
def label_aspects_sentence(text):
    '''Returns a dictionary of aspects in `text` with their corresponding sentences.
    '''
    list_of_aspects = [aspect for aspect in aspect_words if has_aspect(aspect, text)]
    
    aspect_decomposition = {}

    for sentence in re.split('。', text):
        for aspect in list_of_aspects:
            # all we did here is to swap the aspects and the sentences in the key:value pairs
            if aspect not in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[aspect] = [sentence]
            elif aspect in aspect_decomposition and has_aspect(aspect, sentence):
                aspect_decomposition[aspect].append(sentence)
    
    return aspect_decomposition

In [16]:
def compute_aspect_sentiment_score_en(row, translator: GoogleTranslator, analyzer: SentimentIntensityAnalyzer, aspect: str):
    '''Uses the VADER sentiment analyzer to assign each review an 
    averaged sentiment score from all the sentences with a specific aspect `aspect`. 

    This requires instantiations of the following:
    - GoogleTranslator from deep-translator
    - SentimentIntensityAnalyzer object from VADER.

    To be used with `df.apply()`.
    '''

    review_text = row['review_text']

    # extracts all the aspects present in the review
    # together with their corresponding sentences in the review
    aspect_sectence_dict = label_aspects_sentence(review_text)

    ##############################################################################
    # helper function to extract positive and negative sentiment confidence scores
    def get_compound_sentiment(text):
        '''This is a helper function to extract out the positive and negative sentiment confidence levels in `text`.

        Returns
        ---
        A tuple `(pos, neg)`, where `pos` is the positive sentiment confidence, and `neg` is the negative sentiment confidence.
        '''
        sentiment_data = analyzer.polarity_scores(text)

        return sentiment_data['compound']
    ##############################################################################

    if aspect == 'all':
        review_text_en = translator.translate(review_text)
        net_sentiment = get_compound_sentiment(review_text_en)
        return net_sentiment
    
    # checks if the review contains the given aspect
    elif aspect in aspect_sectence_dict:
        sentiments = []
        for sentence in aspect_sectence_dict[aspect]:
            sentence_en = translator.translate(sentence)
            net_sentiment = get_compound_sentiment(sentence_en)
            sentiments.append(net_sentiment)
        
        return np.mean(sentiments)
    
    else: # if aspect is not present in review
        return 0


In [17]:
# helper lookup table for easier population
aspect_lookup_table = {
    'all': 'overall',
    'service': 'service',
    'location': 'location',
    'room': 'room',
    'amenities': 'amenities',
    'bathroom': 'bathroom',
    'food': 'food'
}

In [18]:
label_aspects_sentence('日本酒の飲み比べサーバは良かったです。また、部屋もきれいでスマホ充電など細かな気遣いも良かったです')

{'service': ['日本酒の飲み比べサーバは良かったです'],
 'food': ['日本酒の飲み比べサーバは良かったです'],
 'room': ['また、部屋もきれいでスマホ充電など細かな気遣いも良かったです']}

We shall stop here for now and leave this effort to future work, as the translator takes too long to run for all the texts, and running it on a small sample of text is not really illuminating enough for us to perform any statistical analysis.